# Yolo3 Finetuning with AWS

This series of notebooks demonstrates how to finetune pretrained YOLO v3 (aka YOLO3) using MXNet on AWS.

**This notebook** walks through using the [SageMaker Hyperparameter Tuning Job](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html) tool to finding optmized hypterparameter and finetune the model.

**Follow-on** the content of the notebooks shows:

* How to use MXNet YOLO3 pretrained model
* How to create Ground-Truth dataset from images the model mis-detected
* How to finetune the model using the created dataset
* Load your finetuned model and Deploy Sagemaker-Endpoint with it.
* Apply Elastic Inference to your endpoint.

## Pre-requisites

This notebook is designed to be run in Amazon SageMaker. To run it (and understand what's going on), you'll need:

* Basic familiarity with Python, [MXNet](https://mxnet.apache.org/), [AWS S3](https://docs.aws.amazon.com/s3/index.html), [Amazon Sagemaker](https://aws.amazon.com/sagemaker/)
* To create an **S3 bucket** in the same region, and ensure the SageMaker notebook's role has access to this bucket.
* Sufficient [SageMaker quota limits](https://docs.aws.amazon.com/general/latest/gr/aws_service_limits.html#limits_sagemaker) set on your account to run GPU-accelerated spot training jobs.

## Cost and runtime

Depending on your configuration, this demo may consume resources outside of the free tier but should not generally be expensive because we'll be training on a small number of images. You might wish to review the following for your region:

* [Amazon SageMaker pricing](https://aws.amazon.com/sagemaker/pricing/)

The standard `ml.t2.medium` instance should be sufficient to run the notebooks.

We will use GPU-accelerated instance types for training and hyperparameter optimization, and use spot instances where appropriate to optimize these costs.

As noted in the step-by-step guidance, you should take particular care to delete any created SageMaker real-time prediction endpoints when finishing the demo.

# Step 0: Dependencies and configuration

As usual we'll start by loading libraries, defining configuration, and connecting to the AWS SDKs:

In [1]:
%load_ext autoreload
%autoreload 1

# Built-Ins:
import os
import json
from datetime import datetime
from glob import glob
from pprint import pprint
from matplotlib import pyplot as plt

# External Dependencies:
import boto3
import imageio
import sagemaker
import numpy as np
from sagemaker.mxnet import MXNet
from botocore.exceptions import ClientError

/Users/dongkyl/.pyenv/versions/3.7.4/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Step 1: Get best job informations 

## Step 2: Create Sagemaker Model

In [ ]:
mxnet_model = sagemaker.mxnet.model.MXNetModel(
    name='hyundai-yolo3-deploy',
    model_data=f'{est.output_path}/{est.base_job_name}/output/model.tar.gz', 
    role=role, 
    entry_point='inference.py',
    source_dir='src',
    framework_version='1.4.1',
    py_version='py3',
)

## Step 3: Deploy Model

In [ ]:
predictor = mxnet_model.deploy(
    instance_type='ml.c5.xlarge', initial_instance_count=1,
    accelerator_type='ml.eia2.xlarge'
)

## Step 4: Invoke Sagemaker Endpoint

In [ ]:
with open('soccer.jpg', 'rb') as fp:
    bimage = fp.read()
s = b64encode(bimage).decode('utf-8')

In [ ]:
%%time
res = predictor.predict({
    'short': 416,
    'image': s
})
print(res['shape'])

In [ ]:
ax = viz.plot_bbox(mx.image.imresize(mx.image.imdecode(bimage), 492, 416), mx.nd.array(res['bbox']), mx.nd.array(res['score']), mx.nd.array(res['cid']), class_names=['person'])